# MULTIPLE REGRESSION IN TENSOR FLOW:

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
from collections import Counter
from nltk.tokenize import LineTokenizer, SpaceTokenizer, TweetTokenizer
from nltk import word_tokenize
from nltk import word_tokenize, PorterStemmer, WordNetLemmatizer
import string
import matplotlib
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix, hstack
from sklearn import preprocessing as skpp
from sklearn.model_selection import train_test_split 
import tensorflow as tf


In [ ]:
df_miss_50000=pd.read_pickle('/Users/Desktop/df_50000')

In [3]:
df_miss_50000.head(5)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,sub_cat1,sub_cat2,sub_cat3,text,tokens,Size,MostCommonWord,brand_prediction,confidence_level,brand_name_predicted
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,MLB,10.0,1,No description yet,Men,Tops,T-shirts,MLB Cincinnati Reds T Shirt Size XL Men Tops T...,"[mlb, cincinnati, red, shirt, size, xl, men, t...",[XL],mlb,"[(1, MLB)]",1.000000,MLB
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,iHome,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,Leather Horse Statues Home Home Décor Home Déc...,"[leather, horse, statue, home, home, décor, ho...",[],home,"[(0.8, iHome)]",0.800000,iHome
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,24K GOLD plated rose Women Jewelry Necklaces n...,"[24k, gold, plated, rose, woman, jewelry, neck...",[],24k,"[(0.09999999999999998, 24/7 Comfort Apparel)]",0.100000,24/7 Comfort Apparel
5,5,Bundled items requested for Ruie,3,Women/Other/Other,TomTom,59.0,0,"Banana republic bottoms, Candies skirt with ma...",Women,Other,Other,Bundled items requested for Ruie Women Other O...,"[bundled, item, requested, ruie, woman, nan, b...",[],bottom,"[(0.6666666666666667, TomTom), (0.666666666666...",0.666667,TomTom
9,9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,Lowa,8.0,0,I realized his pants are on backwards after th...,Vintage & Collectibles,Collectibles,Doll,Porcelain clown doll checker pants VTG Vintage...,"[porcelain, clown, doll, checker, pant, vtg, v...",[],clown,"[(0.6, Lowa), (0.6, CLEAN)]",0.600000,Lowa


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
    

X_cat1 = vectorizer.fit_transform(df_miss_50000['sub_cat1'])
X_cat2 = vectorizer.fit_transform(df_miss_50000['sub_cat2'])
# X_cat3 = vectorizer.fit_transform(df_miss_50000['sub_cat3'])
# X_item = vectorizer.fit_transform(df_miss_50000['item_description'])
X_most_common_word=vectorizer.fit_transform(df_miss_50000['MostCommonWord'])
X_dummies = csr_matrix(pd.get_dummies(df_miss_50000[['item_condition_id', 'shipping']],sparse=True).values)

from scipy.sparse import csr_matrix, hstack
# sparse_merge = hstack((X_cat1, X_cat2,X_cat3,X_item,X_most_common_word,X_dummies)).tocsr()
sparse_merge = hstack((X_cat1,X_cat2, X_most_common_word,X_dummies)).tocsr()

sparse_merge=csr_matrix(sparse_merge.shape, dtype=np.float32).toarray()



## reduced the number of categries becoz of slow processing.

In [5]:
X=sparse_merge

In [ ]:
sparse_merge

In [ ]:
type(X)
X.shape


In [6]:
y=df_miss_50000['price'].astype(np.float32)  ##y is the price to be predicted.
y.ndim
y=y.values.reshape(len(y),1)
y

1

array([[10.],
       [35.],
       [44.],
       ...,
       [ 8.],
       [53.],
       [ 8.]], dtype=float32)

y.ndim

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=123)
num_outputs = y_train.shape[1] 
num_inputs = X_train.shape[1]
X_train.shape
num_inputs
num_outputs

(30000, 5779)

5779

1

In [8]:
X_train.shape
type(X_train)
# X_train=X_train.todense()
X_train.shape

(30000, 5779)

numpy.ndarray

(30000, 5779)

In [9]:
X_train.shape
y_train.shape

(30000, 5779)

(30000, 1)

In [10]:
x_tensor = tf.placeholder(dtype=tf.float32, 
    shape=[None, num_inputs], name="x") 
y_tensor = tf.placeholder(dtype=tf.float32, 
    shape=[None, num_outputs], name="y") 

w = tf.Variable(tf.zeros([num_inputs,num_outputs]), 
    dtype=tf.float32, name="w") 
b = tf.Variable(tf.zeros([num_outputs]), 
    dtype=tf.float32, name="b") 

model = tf.matmul(x_tensor, w) + b


In [11]:
loss = tf.reduce_mean(tf.square(model - y_tensor))
# mse and R2 functions
mse = tf.reduce_mean(tf.square(model - y_tensor))
y_mean = tf.reduce_mean(y_tensor)
total_error = tf.reduce_sum(tf.square(y_tensor - y_mean))
unexplained_error = tf.reduce_sum(tf.square(y_tensor - model))
rs = 1 - tf.div(unexplained_error, total_error)

learning_rate = 0.001
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

num_epochs = 1500
loss_epochs = np.empty(shape=[num_epochs],dtype=np.float32)
mse_epochs = np.empty(shape=[num_epochs],dtype=np.float32)
rs_epochs = np.empty(shape=[num_epochs],dtype=np.float32)

mse_score = 0
rs_score = 0

In [14]:
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())

    for epoch in range(num_epochs):
        feed_dict = {x_tensor: X_train, y_tensor: y_train}
        loss_val, _ = tfs.run([loss, optimizer], feed_dict)
        loss_epochs[epoch] = loss_val


        mse_score, rs_score = tfs.run([mse, rs], feed_dict)
        mse_epochs[epoch] = mse_score
        rs_epochs[epoch] = rs_score

print('For test data : MSE = {0:.8f}, R2 = {1:.8f} '.format(
    mse_score, rs_score))

For test data : MSE = 634.16931152, R2 = -0.00169790 


We see that the accuracy is low for this model as well.
We need to optimize the model for better accuracy.
Maybe by changing the number of features or the kind of features.